**Author**: JW Debelius<br>
**License**: BSD-2<br>
**Copyright**: Copyright JW Debelius, 2016

# Introduction

We will start by simulating distributions, power and effect sizes from common distributions.

In [1]:
import os
import pickle

from matplotlib import rcParams
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sn
import scipy.stats
from scipy.stats import norm as z

from power.power import subsample_power, confidence_bound
import utils.simulate as sim
import utils.traditional as trad
import utils.plot as plot
import utils.utils as ap

# Simulation Parameters
We will perform 100 simulations of statistical power, comparing the power between 5 and 50 counts, with 5 observation intervals between. We'll set the critical value at 0.05.

In [6]:
num_rounds = 1000
counts = np.arange(5, 55, 10)
alpha = 0.05
distributions = {}



We'll also set a random seed, so that the results we get out are consistent.

In [7]:
np.random.seed(25)

# Case I T test

We're going to start by looking at a case I t test.

The test checkes if an observation is drawn from a sample. We are testing the alternative hypotheses,

$\begin{matrix}
\textbf{H}_{0} & x = \bar{x} \\
\textbf{H}_{1} & x \neq \bar{x}\\
\end{matrix} \tag{1}$

where $\bar{x}$ is the mean of the population, $x$ is the value being compared to the sample, $s$ is the standard devation of the sample, and there are $n$ observations in the sample.

The test statistic for the case I t test is given as

$t = \frac{(\bar{x} - x)\sqrt{n}}{s} \tag{2}$

The probability distribution follows a T distribution with $n-1$ degrees of freedom, where $n$ is the number of observations in the sample.


For the emperical test, we will use the `scipy.stats.ttest_1samp` function, which returns a p value.

In [8]:
def emp_ttest_1(sample, x0=0):
    """Wraps a one sample t test for power calculation"""
    return scipy.stats.ttest_1samp(sample, x0)[1]

The effect size for a one-sample t test is Cohen's d, given by equation (3).
$\begin{align*}
d = \frac{\bar{x} - x_{0}}{s}
\end{align*}\tag{3}$
For a two-tailed test, we can treat this as an absloute magnitude, and then test $t > (1 - \alpha/2)$. This is implemented and tested in the `traditional` module.

In a parametric test, we look at power for the probability under the alternative hypothesis. The noncentrality parameter is used to modify the alternative hypothesis. For a case I t test, the non centrality paramter for the statistic, $\lambda$ is given by
$\begin{align*}
\lambda &= \frac{t}{\sqrt{n}}\\
&=\frac{1}{\sqrt{n}}\left(\frac{(\bar{x} - x)\sqrt{n}}{s}\right )\\
&=\frac{(\bar{x} - x)}{s}
\end{align*}\tag{4}$

We will encorperate this in the power calculation. To allow for testing, the power calculation has been moved into a library.

Finally, let's define the the parameters we'll use for the simulations.
We'll test the set of hypotheses,

$\begin{matrix}
\textbf{H}_{0} & 0 = \bar{x} \\
\textbf{H}_{1} & 0 \neq \bar{x}\\
\end{matrix} \tag{5}$

where $\bar{x}$ is the mean of a sample drawn from a population.

We'll test the means between 2 and 10 with standard deviations between 5 and 15, and sample sizes between 60 and 100 observations.



In [9]:
distributions['ttest_1'] = {'clean_name': 'One Sample T test',
                            'emp_test_fun': emp_ttest_1,
                            'trad_test_fun': trad.calc_ttest_1,
                            'sim_function': sim.ttest_1_simulate,
                            'sim_parameters': [[2, 10], [5, 15], [60, 100]],
                            'other_sim_parameters': {},
                            'test_parameters': {'x0': 0},
                            'emp_parameters': {},
                            'effect_size': trad.cohen_d_one_sample,
                             }

AttributeError: module 'utils.traditional' has no attribute 'cohen_d_one_sample'

# Independent Sample T test

he case II t test is a test for two independent samples, to determine if the samples are drawn from different distributions.

$\begin{matrix}
\textbf{H}_{0} & \bar{x}_{1} = \bar{x}_{2} \\
\textbf{H}_{1} & \bar{x}_{1} \neq \bar{x}_{2}\\
\end{matrix} \tag{6}$

There are several ways to calculate this t statistic, but we will operate on the assumption that the two populations have different variances, giving the most extensibe calculation of the test statistic. So,

$\begin{align*}
t &= \frac{\bar{x}_{1} - \bar{x}_{2}}{\sqrt{\frac{s_{1}^{2}}{n_{1}} + \frac{s_{2}^{2}}{n_{2}}}}\\
&= \frac{\bar{x}_{1} - \bar{x}_{2}}{\sqrt{\frac{n_{2}s_{1}^{2} + n_{1}s_{2}^{2}}{n_{1}n_{2}}}}
\end{align*}\tag{7}$

The t statistic follows a T distribution with $df$ degrees of freedom, where $df$ is given as
$df = \frac{(s_{1}^{2}/n_{1} + s_{2}^{2}/n_{2})^{2}}{(s_{1}^{2}/n_{1})^2/(n_{1}-1) + s_{2}^{2}/n_{2})^2/(n_{2}-1)} \tag{8}$

For the sake of simplicity, we'll assume that $n_{1} = n_{2}$, which allows us to redefine equation (7) as
$\begin{align*}
t &= \frac{(\bar{x}_{1} - \bar{x}_{2})}{\sqrt{\frac{s_{1}^{2}}{n} + \frac{s_{2}^{2}}{n}}}\\
&= \frac{\sqrt{n}(\bar{x}_{1} - \bar{x}_{2})}{\sqrt{s_{1}^{2} + s_{2}^{2}}}
\end{align*}\tag{9}$
which means the test statitic is now drawn from a t distribution with df degrees of freedom, where
df is defined as
$\begin{align*}
df &= \left (n-1 \right ) \left (\frac{\left (s_{1}^{2} + s_{2}^{2}  \right )^{2}}{\left (s_{1}^{2} \right)^{2} + \left (s_{2}^{2}  \right )^{2}} \right )\\
\end{align*}\tag{10}$

For the emperical test, we can use the `scipy.stats.ttest_ind` function, which will return a p value.

In [8]:
def emp_ttest_ind(sample1, sample2):
    """Wraps a independent sample t test for power calculation"""
    return scipy.stats.ttest_ind(sample1, sample2)[1]

We again use Cohen's d for the effect size, although in this case, $s_{p}$ is substituted into equation (2.3) as the standard deviation, and we're looking at the difference of the two sample means. 
$\begin{align*}
s_{p} &= \sqrt{\frac{\left (n_{1} - 1 \right)s_{1}^{2} + \left (n_{2} - 1 \right)s_{2}^{2}}{n_{1} + n_{2} - 2}}\\
&= \sqrt{s_{1}^{2} + s_{2}^{2}}
\end{align*}\tag{11}$
$\begin{align*}
d = \frac{\bar{x}_{1} - \bar{x}_{2}}{s_{p}}
\end{align*}\tag{12}$

This has been implemented in the `traditional` module to allow for testing.

The effect size, non-centrality parameter, for an independent sample t test where samples are the same size is once again related to the test statistic as
$\begin{align*}
\lambda &= \frac{t}{\sqrt{n}}\\
&= \left (\frac{\sqrt{n} \left (\bar{x}_{1} - \bar{x}_{2} \right )}{\sqrt{s_{1}^{2} + s_{2}^{2}}} \right ) \left (\frac{1}{\sqrt{n}} \right )\\
&= \left (\frac{\bar{x}_{1}^{2} - \bar{x}_{2}^{2}}{\sqrt{s_{1}^{2} + s_{2}^{2}}} \right )
\end{align*}\tag{13}$

We will test distributions with means between 0 and 10, standard deviations between 5 and 15, and samples sizes between 60 and 100 observations.

In [9]:
def ttest_ind_simulate(mu_lim, sigma_lim, counts_lims):
    """Simulates data for an independent sample t test"""
    # Gets the distribution paramters
    mu1, mu2 = np.random.randint(*mu_lim, size=2)
    sigma1, sigma2 = np.random.randint(*sigma_lim, size=2)
    n = np.random.randint(*counts_lims)
    
    # Returns a pair of distributions
    samples =  [mu1 + np.random.randn(n) * sigma1, mu2 + np.random.randn(n) * sigma2]
    return [mu1, mu2, sigma1, sigma2, n], samples

In [10]:
distributions['ttest_ind'] = {'clean_name': 'Two Sample T test',
                              'emp_parameters': {},
                              'emp_test_fun': emp_ttest_ind,
                              'other_sim_parameters': {},
                              'sim_function': ttest_ind_simulate,
                              'sim_parameters': [[0, 10], [5, 15], [60, 100]],
                              'test_parameters': {},
                              'trad_test_fun': trad.calc_ttest_ind,     
                              'effect_size': trad.cohen_ttest_ind
                              }

# One way Analysis of Variance

ssume there exist a set of samples, $\{S_{1}, S_{2}, ..., S_{k} \}$ where there are a total of $N$ observations distributed over the $k$ groups. The $i$th sample, $S_{i}$ contains $n_{i}$ observations, and has a mean of $\bar{x}_{.i}$ and a standard deviation, $s_{i}$ where

$\begin{align*}
s_{i} = \sqrt{\frac{\sum_{j=1}^{n}{\left (x_{ij} - \bar{x}_{.i} \right)^{2}}}{n_{i}-1}}
\end{align*}\tag{14}$

A one-way Analysis of Variance (ANOVA) tests that at least one sample mean in a set of three or more are not equal. Assume that 

$\begin{matrix}
\textbf{H}_{0} & \bar{x}_{1} = \bar{x}_{2} = ... \bar{x}_{k} & \\
\textbf{H}_{1} & \bar{x}_{i} \neq \bar{x}_{j} & \exists i,j \epsilon [1, k], i \neq j
\end{matrix} \tag{15}$

The test statistic for ANOVA is given by
$\begin{align*}
F &= \frac{\frac{\textrm{SS}_{\textrm{between}}}{\textrm{DF}_{\textrm{between}}}}{\frac{\textrm{SS}_{\textrm{within}}}{\textrm{DF}_{\textrm{within}}}}
\end{align*}\tag{16}$

where
$\begin{align*}
\textrm{SS}_{\textrm{total}} &= \sum_{i=1}^{k}{\sum_{j=1}^{n_{i}}{\left (x_{ij} - \bar{x}_{..} \right )^{2}}}\\
\textrm{SS}_{\textrm{between}} &= \sum_{i=1}^{k}{n_{i}\left (\bar{x}_{.i} - \bar{x}_{..} \right )^{2}}\\
\textrm{SS}_{\textrm{within}} &= \sum_{i=1}^{k}{\left [ \sum_{j=1}^{n_{i}}{\left (x_{ij} - \bar{x}_{.i} \right )^{2}}\right ]}
\end{align*}\tag{17}$

$\begin{align*}
\textrm{DF}_{\textrm{total}} &= N - 1\\
\textrm{DF}_{\textrm{between}} &= k - 1\\
\textrm{DF}_{\textrm{within}} &= N - k
\end{align*}\tag{18}$

$\begin{align*}
\textrm{SS}_{\textrm{total}} = \textrm{SS}_{\textrm{between}} + \textrm{SS}_{\textrm{within}}
\end{align*}\tag{19}$

$\begin{align*}
\textrm{DF}_{\textrm{total}} = \textrm{DF}_{\textrm{between}} + \textrm{DF}_{\textrm{within}}
\end{align*}\tag{20}$

and test statistic is drawn from an $F$ distribution with $k - 1$ and $N - k$ degrees of freedom [[1](#Zar)].

In [11]:
def emp_anova(*samples):
    """Wraps a one way ANOVA for power calculation"""
    return scipy.stats.f_oneway(*samples)[1]

For an analysis of variance, the effect size is given by
$\begin{align*}
f &= \sqrt{\frac{\sum_{i=1}^{k}{\left (\bar{x}_{.i} - \bar{x}_{..} \right )^{2}}}{ks^2}}\\
\end{align*}\tag{21}$

Under the alternatively hypothesis, the non-centrality $F'$ is given by

$\begin{align*}
F' = \left(\frac{\textrm{SS}_{\textrm{between}}}{\textrm{SS}_{\textrm{within}}} \right) \left (\frac{\textrm{DF}_{\textrm{within}}}{\textrm{DF}_{\textrm{between}}}{} \right )
\end{align*}\tag{22}$

For a given pair of hypotheses, the noncentrality parameter is defined according to equation (2.4), where the grand mean can be substituted for the the test mean. The overall effect size is therefore given as
$\begin{align*}
\lambda &= \sum_{i=1}^{k}{\lambda_{i}^{2}}\\
&= \sum_{i=1}^{k}\left (\frac{\bar{x}_{i} - \bar{x}_{..}}{s_{i}} \right )^{2} 
\end{align*} \tag{23}$

We will simulate an ANOVA between 3 samples with means between 0 and 10, a common standard deviation between 5 and 15, and between 60 and 100 observations per sample.

In [10]:
def anova_simulation(mu_lim, sigma_lim, count_lims, num_pops):
    """Simulates data for a one way ANOVA"""
    # Defines the distribtuion parameters
    mus = np.random.randint(*mu_lim, size=num_pops)
    sigma = np.random.randint(*sigma_lim)
    n = np.random.randint(*count_lims)
    
    # Draws samples which fit the population
    samples = [mu + np.random.randn(n)*sigma for mu in mus]
    
    return [mus, sigma, n], samples

In [12]:
params, samples = anova_simulation([0, 5], [1, 2], [10, 11], 3)
samples

[array([ 0.865307  ,  2.42238577,  2.9097923 ,  1.20446501, -0.41135187,
         2.82642959,  3.61812261,  3.09582737,  0.595979  ,  1.00288098]),
 array([-0.23554384,  0.39396689,  2.26449758, -0.38587507, -0.45530388,
         0.84384069, -0.89901421,  2.26185087,  1.16407426,  0.54217753]),
 array([ 3.57241728,  3.44345288,  4.0539743 ,  6.47170092,  4.77829176,
         5.48086408,  4.24584553,  4.83578695,  4.59053566,  3.96303158])]

In [13]:
np.concatenate(samples).mean()

2.3020136519395651

In [17]:
np.sum(np.square(np.hstack(samples) - np.concatenate(samples).mean()))

117.73006792127138

In [13]:
distributions['anova'] = {'clean_name': "One way ANOVA",
                          'emp_parameters': {},
                          'emp_test_fun': emp_anova,
                          'other_sim_parameters': {'num_pops': 3},
                          'sim_function': anova_simulation,
                          'sim_parameters': [[0, 10], [5, 15], [60, 100]],
                          'test_parameters': {},
                          'trad_test_fun': trad.calc_anova,
                          'effect_size': trad.cohen_f2
                           }

# Pearsons Correlation Coeffecient

Pearson's correlation coeffecient looks for a linear one-to-one relationship between two vectors, $x$ and $y$, both of size $n$. Closely related vectors have a correlation coeffecient with an absloute value of 1, unrelated data have a correlation coeffecient of 0.

The correlation coeffecient between the two vectors is given by
$\begin{align*}
r = \frac{\sum{xy}}{\sqrt{\sum{x^{2}}\sum{y^{2}}}}
\end{align*}\tag{25}$

We can test the hypotheses,
$\begin{matrix}
\textbf{H}_{0} & r = 0 \\
\textbf{H}_{1} & r \neq 0\\
\end{matrix} \tag{26}$
with a test statistic drawn from the $t$ distribution with $n - 2$ degrees of freedom. The statistic is calculated as
$\begin{align*}
t = \frac{r\sqrt{n-2}}{\sqrt{1 - r^{2}}}
\end{align*}\tag{27}$

Scipy's `scipy.stats.pearsonr` can calculate the correlation coeffecient *and* a p value for the coeffecient.

In [14]:
def emp_pearson(*samples):
    """Wraps pearsons correlation coeffecient for power testing"""
    return scipy.stats.pearsonr(*samples)[1]

It would be nice to have an effect size?

The noncentrality parameter for pearson's correaltion coeffecient is given by
$\begin{align}
\lambda = \frac{r_{xy}\sqrt{n}}{\sqrt{1 - r_{xy}^{2}}}
\end{align}\tag{28}$

And the data is drawn from a t distribution.

The noncentrality parameter and power have been implemented in the `traditional` module.

We're going to simulate regressions with slopes between 1 and 5, intercepts between -2 and 2, and normally distributed residuals with standard deviations between 10 and 100. There will be between 60 and 100 observations in each of the samples. We'll use `regress_simulate` to generate the distributions.

In [15]:
def regress_simulate(mu_lim, sigma_lim, count_lims, b_lims):
    """Simulates data for a one way ANOVA"""
    # Calculates the distribution for the residuals
    sigma = np.random.randint(*sigma_lim)
    n = np.random.randint(*count_lims)
    # Calculates the parameters for the line
    m = np.random.randint(*mu_lim)
    b = np.random.randint(*b_lims)
    
    x = np.arange(-n, n, 2)
    y = m*x + b + np.random.randn(n)*sigma
    
    return [sigma, n, m, b], [x, y]

In [16]:
distributions['pearson'] = {'clean_name': "Pearson's R",
                            'emp_parameters': {'draw_mode': 'matched'},
                            'emp_test_fun': emp_pearson,
                            'other_sim_parameters':{'b_lims':[-2, 2]},
                            'sim_function': regress_simulate,
                            'sim_parameters': [[1, 5], [60, 100], [60, 100]],
                            'test_parameters': {},
                            'trad_test_fun': trad.calc_pearson,
                            }

# Simulations

Now, we're going to build the simulations using the distribution set up described here.

In [56]:
if not os.path.exists('simulations/normal'):
    os.makedirs('simulations/normal')

for k, params in distributions.iteritems():
    print k
    results = {'clean_name': params['clean_name'],
               }
    for i in xrange(100):
        # Calculates the samples
        pop_params, samples = params['sim_function'](*params['sim_parameters'], 
                                                     **params['other_sim_parameters'])
        # Calculates the traditional power
        trad_power = params['trad_test_fun'](*samples, 
                                             counts=counts, 
                                             **params['test_parameters'])
        #  Calculates the emperical power
        empr_power, empr_counts = subsample_power(
            lambda x: params['emp_test_fun'](*x, **params['test_parameters']),
            samples,
            min_counts=5,
            max_counts=55,
            counts_interval=10,
            num_runs=3,
            num_iter=100,
            **params['emp_parameters']
            )
        # calculates the effect sizes
        if 'effect_size' in params:
            base_eff = params['effect_size'](*samples)
        else:
            base_eff = None

        # Updates the information
        results[i] = {'pop_params': pop_params,
                      'samples': samples,
                      'test_power': trad_power,
                      'emprical_power': empr_power,
                      'emprical_counts': empr_counts,
                      'base_effect': base_eff,
                     }
        
    filename = os.path.join('simulations/normal', '%s.p' % k)
    with open(filename, 'w') as f_:
        pickle.dump(results, f_)

pearson
ttest_ind
ttest_1
anova
